# 😷 Masked Face Recognition Training Notebook
This notebook trains both CNN and Transformer (EfficientNetB0) models for 4 tasks: emotion, age, gender, ethnicity.
It is configured to run **100 epochs** for each task.

In [1]:
import os, json, pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [4]:
def build_cnn(num_classes, input_shape=(64,64,3)):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_transformer(num_classes, input_shape=(128,128,3)):
    try:
        base = tf.keras.applications.EfficientNetB0(
            input_shape=input_shape,
            include_top=False,
            weights="imagenet"
        )
    except Exception as e:
        print(" Falling back to random initialization:", e)
        base = tf.keras.applications.EfficientNetB0(
            input_shape=input_shape,
            include_top=False,
            weights=None
        )

    # Freeze most layers
    for layer in base.layers[:-20]:
        layer.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNetB0_Finetuned")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


def prepare_data(df, label_col, img_size=(64,64)):
    X = np.array([np.fromstring(p, sep=" ").reshape(48,48) for p in df['pixels']])
    X = np.expand_dims(X, -1)
    X = np.repeat(X, 3, axis=-1)
    X = np.array([tf.image.resize(img, img_size).numpy() for img in X])
    X = X.astype("float32") / 255.0
    labels = df[label_col].values
    le = LabelEncoder()
    y = to_categorical(le.fit_transform(labels))
    classes = le.classes_
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), classes

def train_and_save(model, X_train, y_train, X_val, y_val,
                   model_name, classes, epochs=15, batch_size=64):
    callbacks = [
        ReduceLROnPlateau(monitor="val_accuracy", factor=0.7, patience=3, verbose=1),
        EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True, verbose=1),
        ModelCheckpoint(f"models/{model_name}.keras", save_best_only=True, monitor="val_accuracy", verbose=1)
    ]
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        callbacks=callbacks
    )
    os.makedirs("models", exist_ok=True)
    with open(f"models/{model_name}_classes.pkl", "wb") as f:
        pickle.dump(list(classes), f)
    with open(f"models/{model_name}_history.json", "w") as f:
        json.dump(history.history, f)
    print(f"Saved {model_name}")
    return history

In [6]:
# =========================
# CNN Training for All Tasks (50 epochs each)
# =========================

# Load dataset (update with your CSV path)
df = pd.read_csv("dataset/masked_face_dataset.csv")

tasks = {
    "emotion": "emotion_label",
    "age": "age_range",
    "gender": "gender",
    "ethnicity": "ethnicity"
}

for task, label_col in tasks.items():
    print(f"\n Training {task} (CNN)")
    (X_train, y_train), (X_val, y_val), (X_test, y_test), classes = prepare_data(df, label_col, img_size=(64,64))
    cnn = build_cnn(len(classes), input_shape=(64,64,3))
    train_and_save(cnn, X_train, y_train, X_val, y_val, f"{task}", classes, epochs=50, batch_size=64)



 Training emotion (CNN)
Epoch 1/50


d:\masked_face_emotion_project_light\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1752 - loss: 1.9305
Epoch 1: val_accuracy improved from None to 0.31857, saving model to models/emotion.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.2183 - loss: 1.8781 - val_accuracy: 0.3186 - val_loss: 1.7258 - learning_rate: 0.0010
Epoch 2/50
262/263 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.3318 - loss: 1.7023
Epoch 2: val_accuracy improved from 0.31857 to 0.37238, saving model to models/emotion.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - accuracy: 0.3541 - loss: 1.6553 - val_accuracy: 0.3724 - val_loss: 1.5826 - learning_rate: 0.0010
Epoch 3/50
262/263 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.4368 - loss: 1.4843
Epoch 3: val_accuracy improved from 0.37238 to 0.48810, saving model to models/emotion.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.4530 - loss: 1.4511 - val_accuracy: 0.4881 - val_loss: 1.3540 - learning_rate: 0.0010
Epoch 4/50
262/263 ━━━━━━━━━━━━━━━━━━━━ 0s 78

d:\masked_face_emotion_project_light\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.6058 - loss: 0.8046
Epoch 1: val_accuracy improved from None to 0.69571, saving model to models/age.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 31s 111ms/step - accuracy: 0.6467 - loss: 0.7304 - val_accuracy: 0.6957 - val_loss: 0.6526 - learning_rate: 0.0010
Epoch 2/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.7089 - loss: 0.6289
Epoch 2: val_accuracy improved from 0.69571 to 0.74095, saving model to models/age.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 29s 110ms/step - accuracy: 0.7099 - loss: 0.6222 - val_accuracy: 0.7410 - val_loss: 0.5832 - learning_rate: 0.0010
Epoch 3/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.7430 - loss: 0.5623
Epoch 3: val_accuracy improved from 0.74095 to 0.77000, saving model to models/age.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 29s 111ms/step - accuracy: 0.7496 - loss: 0.5495 - val_accuracy: 0.7700 - val_loss: 0.5128 - learning_rate: 0.0010
Epoch 4/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 

d:\masked_face_emotion_project_light\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.8034 - loss: 0.4819
Epoch 1: val_accuracy improved from None to 0.83048, saving model to models/gender.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 56s 194ms/step - accuracy: 0.8108 - loss: 0.4306 - val_accuracy: 0.8305 - val_loss: 0.3754 - learning_rate: 0.0010
Epoch 2/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.8282 - loss: 0.3655
Epoch 2: val_accuracy improved from 0.83048 to 0.86524, saving model to models/gender.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 32s 122ms/step - accuracy: 0.8387 - loss: 0.3457 - val_accuracy: 0.8652 - val_loss: 0.3039 - learning_rate: 0.0010
Epoch 3/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.8705 - loss: 0.2870
Epoch 3: val_accuracy improved from 0.86524 to 0.88429, saving model to models/gender.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 29s 110ms/step - accuracy: 0.8767 - loss: 0.2726 - val_accuracy: 0.8843 - val_loss: 0.2865 - learning_rate: 0.0010
Epoch 4/50
263/263 ━━━━━━━━━━━━

d:\masked_face_emotion_project_light\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.7400 - loss: 0.8435
Epoch 1: val_accuracy improved from None to 0.75524, saving model to models/ethnicity.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 48s 177ms/step - accuracy: 0.7527 - loss: 0.8011 - val_accuracy: 0.7552 - val_loss: 0.7254 - learning_rate: 0.0010
Epoch 2/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.7490 - loss: 0.7330
Epoch 2: val_accuracy improved from 0.75524 to 0.77286, saving model to models/ethnicity.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 35s 131ms/step - accuracy: 0.7583 - loss: 0.7005 - val_accuracy: 0.7729 - val_loss: 0.6289 - learning_rate: 0.0010
Epoch 3/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.7671 - loss: 0.6351
Epoch 3: val_accuracy improved from 0.77286 to 0.79524, saving model to models/ethnicity.keras
263/263 ━━━━━━━━━━━━━━━━━━━━ 30s 112ms/step - accuracy: 0.7745 - loss: 0.6092 - val_accuracy: 0.7952 - val_loss: 0.5545 - learning_rate: 0.0010
Epoch 4/50
263/263 ━━━

In [5]:
# =========================
# Transformer Training for All Tasks (50 epochs each)
# =========================

for task, label_col in tasks.items():
    print(f"\n Training {task} (Transformer)")
    (X_train, y_train), (X_val, y_val), (X_test, y_test), classes = prepare_data(df, label_col, img_size=(128,128))
    transf = build_transformer(len(classes), input_shape=(128,128,3))
    train_and_save(transf, X_train, y_train, X_val, y_val, f"{task}_transformer", classes, epochs=50, batch_size=32)



 Training emotion (Transformer)
 Falling back to random initialization: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)
Epoch 1/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.1473 - loss: 2.0208
Epoch 1: val_accuracy improved from None to 0.14286, saving model to models/emotion_transformer.keras
525/525 ━━━━━━━━━━━━━━━━━━━━ 104s 181ms/step - accuracy: 0.1432 - loss: 1.9834 - val_accuracy: 0.1429 - val_loss: 1.9470 - learning_rate: 1.0000e-04
Epoch 2/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.1482 - loss: 1.9494
Epoch 2: val_accuracy improved from 0.14286 to 0.14571, saving model to models/emotion_transformer.keras
525/525 ━━━━━━━━━━━━━━━━━━━━ 92s 175ms/step - accuracy: 0.1444 - loss: 1.9488 - val_accuracy: 0.1457 - val_loss: 1.9477 - learning_rate: 1.0000e-04
Epoch 3/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.1393 - loss: 1.9462
Epo

In [7]:
import streamlit as st
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
from utils import preprocess_image

st.set_page_config(page_title=" Masked Face Recognition", layout="wide")

st.markdown(
    """
    <style>
        .title {
            font-size: 36px;
            font-weight: bold;
            text-align: center;
            color: #4CAF50;
        }
        .subtitle {
            text-align: center;
            font-size: 18px;
            color: #888;
        }
        .result-card {
            padding: 20px;
            border-radius: 15px;
            background-color: #f8f9fa;
            margin-bottom: 15px;
            box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
        }
    </style>
    """,
    unsafe_allow_html=True,
)

st.markdown('<div class="title">Masked Face Attribute Recognition</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Predict Emotion, Age Range, Gender & Ethnicity</div>', unsafe_allow_html=True)


def load_model_and_labels(model_path, labels_path):
    try:
        model = load_model(model_path)
        with open(labels_path, "rb") as f:
            labels = pickle.load(f)
        return model, labels
    except Exception as e:
        st.error(f"Error loading {model_path}: {e}")
        return None, []

cnn_emotion, EMOTIONS   = load_model_and_labels("models/emotion.keras", "models/emotion_classes.pkl")
cnn_age, AGES           = load_model_and_labels("models/age.keras", "models/age_classes.pkl")
cnn_gender, GENDERS     = load_model_and_labels("models/gender.keras", "models/gender_classes.pkl")
cnn_ethnicity, ETHNICITY= load_model_and_labels("models/ethnicity.keras", "models/ethnicity_classes.pkl")

AGE_MAPPING = {
    "0": "0–9", "1": "10–19", "2": "20–29", "3": "30–39",
    "4": "40–49", "5": "50–59", "6": "60–69", "7": "70+"
}

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def predict_attributes(img):
    img_processed = preprocess_image(img)  
    img_processed = np.expand_dims(img_processed, axis=0)

    results = {}

    if cnn_emotion:
        pred = cnn_emotion.predict(img_processed, verbose=0)
        results["Emotion"] = (EMOTIONS[np.argmax(pred)], float(np.max(pred)))

    if cnn_age:
        pred = cnn_age.predict(img_processed, verbose=0)
        raw_label = str(AGES[np.argmax(pred)])
        results["Age Range"] = (AGE_MAPPING.get(raw_label, raw_label), float(np.max(pred)))

    if cnn_gender:
        pred = cnn_gender.predict(img_processed, verbose=0)
        results["Gender"] = (GENDERS[np.argmax(pred)], float(np.max(pred)))

    if cnn_ethnicity:
        pred = cnn_ethnicity.predict(img_processed, verbose=0)
        results["Ethnicity"] = (ETHNICITY[np.argmax(pred)], float(np.max(pred)))

    return results

tab1, tab2 = st.tabs(["Upload Image", "Live Webcam"])

# ===== Upload Tab =====
with tab1:
    uploaded_file = st.file_uploader("Upload a face image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        file_name = uploaded_file.name.lower() 
        file_bytes = np.frombuffer(uploaded_file.read(), np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) > 0:
            (x,y,w,h) = faces[0]
            face = img[y:y+h, x:x+w]

            results = predict_attributes(face)

            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            st.image(img, caption="Detected Face", channels="BGR")

            st.subheader("🔮 Predictions")
            for key, (label, conf) in results.items():
                st.markdown(
                    f'<div class="result-card"><b>{key}</b>: {label}</div>',
                    unsafe_allow_html=True
                )
                st.progress(conf)

        else:
            st.warning(" No face detected!")


with tab2:
    st.write("Enable webcam to see live predictions in real-time")
    run = st.checkbox("Start Webcam")
    FRAME_WINDOW = st.image([])
    results_placeholder = st.empty()

    cap = cv2.VideoCapture(0)
    cap.set(3, 320)
    cap.set(4, 240)

    frame_skip = 3
    count = 0

    while run:
        ret, frame = cap.read()
        if not ret:
            st.error("Failed to capture image")
            break

        frame = cv2.flip(frame, 1)
        count += 1

        if count % frame_skip == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            if len(faces) > 0:
                (x,y,w,h) = faces[0]
                face = frame[y:y+h, x:x+w]
                results = predict_attributes(face)

                cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

                with results_placeholder.container():
                    st.subheader("🔮 Live Predictions")
                    for key, (label, conf) in results.items():
                        st.markdown(
                            f'<div class="result-card"><b>{key}</b>: {label}</div>',
                            unsafe_allow_html=True
                        )
                        st.progress(conf)

        FRAME_WINDOW.image(frame, channels="BGR")

    cap.release()


2025-09-01 19:42:06.573 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 19:42:06.574 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 19:42:07.002 
  command:

    streamlit run d:\masked_face_emotion_project_light\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-01 19:42:07.003 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 19:42:07.004 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 19:42:07.005 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 19:42:07.006 Thread 'MainThread': missing ScriptRunContext! This warning can be